# Deep State - Local version

In [1]:
import gluonts
gluonts.__version__

'0.4.1'

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
import pprint

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [6]:
def deep_state(seed=42, data="m4_quarterly", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)

    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,

    )

    predictor = estimator.train(dataset.train)
    
#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    
    df=pd.DataFrame([output])
#     pprint(output)
    
    return df

In [7]:
%%time
if __name__ == "__main__":
    df = deep_state(seed=42, data="m4_weekly", epochs=1, batches=1)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1/1 [00:11<00:00, 11.67s/it, avg_epoch_loss=0.625]
INFO:root:Epoch[0] Elapsed time 11.698 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.624855
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 0.6248546242713928 (occurred at epoch 0)
INFO:root:End model training
Running evaluation: 100%|██████████| 359/359 [02:14<00:00,  2.66it/s] 


Wall time: 2min 27s


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],epochs,seed
0,13.121585,0.179142,145.712068,0.162294,0.129815,1,42
